In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime


# Create dependent variables


### 1. Create list of tickers

In [2]:
tickers = ["BTC-USD", "SPY", "GLD", "DX-Y.NYB"]


###  2. Create data frame to consist of weekly return for each ticker


[Retrieve financial data from yahoo finance](https://pypi.org/project/yfinance/)

[Deal with multi-level columns](https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801)

[Resample data using pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html)

[Function pct_change()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html)

In [3]:
# Retrieve daily data from yahoo finance from March 7, 2017 to September 25, 2022 (5 years)
# Resample into weekly data bases on average of that week
# Calculate percent change from last week and multiply by 100 to get (%) unit 
data = yf.download(tickers, 
                   start='2017-7-3', 
                   end='2022-9-25').resample('W').mean().pct_change(periods=1)*100 

# Take only the closing price
data = data['Close']   

#print(data)



[*********************100%***********************]  4 of 4 completed


## 3. Calculate rolling 12 weeks covariance between tickers

[Accessing the index in 'for' loops](https://stackoverflow.com/questions/522563/accessing-the-index-in-for-loops)

[Calculate rolling covariance: pandas.DataFrame.rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)

[Format string literal](https://stackoverflow.com/questions/57150426/what-is-printf)

[Selecting multiple columns in a Pandas dataframe](https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe/11287278#11287278)

In [4]:
for i, ticker1 in enumerate(tickers[:-1]):      # function enumerate() iterates through both index and value 
                                                # of the element in the list
    for ticker2 in tickers[i+1:]:       # Loop through the list starting from the next ticker to ticker 1
        data[f"{ticker1}-{ticker2}"] = data[ticker1].rolling(12).cov(data[ticker2])
 #name the column using formatted string literal 
                                       # Calculate rolling covariance between two tickers using 12-week window
                                       # Rolling function uses up to 11 weeks prior to calculate
        data = data.iloc[ : , :7]      # Update data frame to include only first 7 columns

#print(data)
        
        


# Create independent variable

2 steps:
- Step 1: Collect data from web or csv files
- Step 2: Combine data frames

[Import csv columns into data frame](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

[Combine 2 data frame using concatnate](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)


### 1. Collect CBOE VIX data from yahoo finance

In [5]:
# Download data from yahoo finance
data2 = yf.download('^VIX',
                     start='2017-7-3', 
                     end='2022-9-25').resample('W').mean()
data2 = data2['Close']

#print(data4.head(15))

[*********************100%***********************]  1 of 1 completed


In [6]:
# Combine VIX into existing data frame
data3 = pd.concat([data, data2], axis=1)

data3.index = data3.index.date              # Extract the date part from datetime index
data3.index = pd.to_datetime(data3.index)   # Convert the date index back into datetime format
data3 = data3.groupby(data3.index).first()  # Group by date and only take the first row
 
#print(data3)

### 2. Collect Bitcoin google trends and GARCH(1,1) volatility from csv files



In [39]:
# Import specific columns from csv file
df_csv = pd.read_csv('/Users/quanghuy/Documents/ECO 590 - R & Python/Nguyen_Huy_Data.csv', on_bad_lines= 'skip', 
                     delimiter = ';', usecols = ['time','gg_trend_wrld','btc_garch'], index_col = 'time')

# Reformat the date column to match the data frame in python
df_csv.index = pd.to_datetime(df_csv.index, format='%d/%m/%Y', utc = True).strftime('%Y-%m-%d')
        
# Convert the date column to datetime format
df_csv.index = pd.to_datetime(df_csv.index)



In [46]:
# Combine data frames
data4 = pd.concat([data3, df_csv], axis=1)

# Replace '.' value with none
data4 = data4.replace('.', None)    

# Fix wrong decimal seperator, from ',' to '.'. And convert the 'btc_garch' column to numeric value
data4['btc_garch'] = data4['btc_garch'].str.replace(',', '.').astype(float)

#  Convert the 'gg_trend_wrld' column to numeric value
data4['gg_trend_wrld'] = data4['gg_trend_wrld'].astype(float)

print(data4.dtypes)
data4.head(15)

BTC-USD             float64
DX-Y.NYB            float64
GLD                 float64
SPY                 float64
BTC-USD-SPY         float64
BTC-USD-GLD         float64
BTC-USD-DX-Y.NYB    float64
Close               float64
gg_trend_wrld       float64
btc_garch           float64
dtype: object


,BTC-USD,DX-Y.NYB,GLD,SPY,BTC-USD-SPY,BTC-USD-GLD,BTC-USD-DX-Y.NYB,Close,gg_trend_wrld,btc_garch
2017-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5050,NaN,53.627670
2017-07-16,-13.097695,-0.412155,-0.127486,0.744076,NaN,NaN,NaN,10.3420,NaN,73.762044
2017-07-23,14.188202,-1.174701,1.944076,1.116904,NaN,NaN,NaN,9.6880,NaN,128.616841
2017-07-30,5.483454,-0.841793,1.287690,0.293791,NaN,NaN,NaN,9.7720,NaN,97.603705
2017-08-06,8.746409,-0.799882,0.576355,0.009713,NaN,NaN,NaN,10.2200,NaN,82.276999
2017-08-13,22.760548,0.427892,0.604611,-0.456351,NaN,NaN,NaN,12.7100,NaN,83.889070
2017-08-20,18.010030,0.169143,0.640603,-0.363337,NaN,NaN,NaN,13.1840,NaN,69.232944
2017-08-27,0.128063,-0.433899,0.510203,-0.354057,NaN,NaN,NaN,12.0600,NaN,53.292177
2017-09-03,8.717764,-0.622560,2.069630,0.762210,NaN,NaN,NaN,10.9920,NaN,55.632517
2017-09-10,-5.874469,-0.776590,2.016871,0.183015,NaN,NaN,NaN,11.8825,NaN,62.425078


### 3. Create time dummy variable for covid period

In [48]:
data4['covid'] = 0

start = '2020-01-01'
end = '2020-08-31'

requirement = (data4.index >= start) & (data4.index <= end)
data4.loc[requirement,'covid'] = 1

#data4[requirement]


# Export data frame into csv file

[Rename column names](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename)

[Python to csv file](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)



In [49]:
# Rename columns
data4 = data4.rename(columns ={'BTC-USD': 'r_btc(%)',
                           'DX-Y.NYB': 'r_usd(%)', 
                           'GLD': 'r_gld(%)',
                           'SPY': 'r_spy(%)',
                           'BTC-USD-SPY': 'cov_bspy',
                           'BTC-USD-GLD': 'cov_bgld',
                           'BTC-USD-DX-Y.NYB': 'cov_busd',
                           'Close': 'VIX'})


In [51]:

# Export to csv file
data4.to_csv('Nguyen_Huy_Dataa.csv', index=True, index_label = 'Date')  
